In [11]:
from vizdoom import *
import random
import time
import sys

In [12]:
# Setup game
game = DoomGame()
game.load_config('github/VizDoom/scenarios/basic.cfg')
game.init()

In [13]:
import numpy as np

actions = np.identity(3, dtype=np.uint8)
actions

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [14]:
random.choice(actions)

array([0, 1, 0], dtype=uint8)

In [15]:
game.make_action(random.choice(actions))

-1.0

In [17]:
episodes = 10
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        
        
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: 99.0
Result: 79.0
reward: -4.0
reward: 99.0
Result: 95.0
reward: -4.0
reward: 99.0
Result: 95.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0

In [11]:
game.close()

In [13]:
game.get_state().screen_buffer.shape

(3, 240, 320)

# Converting it to a Gym Environment

In [17]:
from gym import Env
from gym.spaces import Discrete, Box
import cv2

In [18]:
# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config('github/VizDoom/scenarios/basic.cfg')
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(3)
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [19]:
env = VizDoomGym(render=True)

In [20]:
state = env.reset()

# View Game State

In [21]:
from stable_baselines3.common import env_checker

In [22]:
env_checker.check_env(env)

# View State

In [23]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

# Setup Callback

In [35]:
import os 
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [36]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [37]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [38]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train


In [14]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

In [31]:
env = VizDoomGym(render=True)

In [32]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [39]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_basic\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30       |
|    ep_rew_mean     | -66.8    |
| time/              |          |
|    fps             | 29       |
|    iterations      | 1        |
|    time_elapsed    | 70       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 29.5         |
|    ep_rew_mean          | -60.4        |
| time/                   |              |
|    fps                  | 23           |
|    iterations           | 2            |
|    time_elapsed         | 172          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0064939223 |
|    clip_fraction        | 0.165        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.6        |
|    ep_rew_mean          | -19.5       |
| time/                   |             |
|    fps                  | 20          |
|    iterations           | 11          |
|    time_elapsed         | 1115        |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.016191462 |
|    clip_fraction        | 0.218       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.812      |
|    explained_variance   | 0.722       |
|    learning_rate        | 0.0001      |
|    loss                 | 1.17e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | 0.0106      |
|    value_loss           | 2.68e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 16    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.71        |
|    ep_rew_mean          | 83.2        |
| time/                   |             |
|    fps                  | 19          |
|    iterations           | 21          |
|    time_elapsed         | 2230        |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.041776195 |
|    clip_fraction        | 0.254       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.422      |
|    explained_variance   | 0.561       |
|    learning_rate        | 0.0001      |
|    loss                 | 89.3        |
|    n_updates            | 200         |
|    policy_gradient_loss | 0.0411      |
|    value_loss           | 302         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 5.72    

KeyboardInterrupt: 

In [40]:
env.close()